In [52]:
from pathlib import Path
import pandas as pd
import numpy as np

In [53]:
pd.set_option('display.max_columns', 1000)

In [54]:
# Defining directories
dir_path = Path.cwd().parent
prep_folder = dir_path.joinpath("data/prep")
s1_path = prep_folder.joinpath("sent2.parquet")

processed_folder = dir_path.joinpath("data/processed")
if not processed_folder.exists():
    processed_folder.mkdir(parents=True)

In [55]:
df = pd.read_parquet(s1_path)
df

,date,long,lat,species_names,B11,B02,B12,B08,B04,B03
0,2019-08-23,76.98762370000003,31.571770300000026,chir pine,2234.0,376.0,1084.0,3340.0,468.0,739.0
1,2019-08-23,76.98770870000004,31.571733300000062,chir pine,2234.0,240.0,1084.0,3374.0,248.0,496.0
2,2019-08-23,76.98786920000003,31.571645600000068,chir pine,1826.0,269.0,823.0,3458.0,216.0,525.0
3,2019-08-23,76.98792120000007,31.572662200000025,chir pine,2037.0,345.0,971.0,3754.0,359.0,699.0
4,2019-08-23,76.98797090000005,31.57163410000004,chir pine,1826.0,247.0,823.0,3248.0,245.0,529.0
...,...,...,...,...,...,...,...,...,...,...
12135677,2021-11-25,78.12497830000007,31.25865170000003,rhododendron,1781.0,1115.0,1436.0,1827.0,1196.0,1221.0
12135678,2021-11-25,78.12522000000007,31.258880000000033,rhododendron,1657.0,1175.0,1431.0,1704.0,1216.0,1230.0
12135679,2021-11-25,78.12614830000007,31.258660000000077,rhododendron,1678.0,1113.0,1389.0,1954.0,1222.0,1175.0
12135680,2021-11-25,78.12618260000005,31.25882800000005,rhododendron,1966.0,1104.0,1481.0,2143.0,1240.0,1219.0


In [56]:
for band in ["B11", "B02", "B12", "B08", "B04", "B03"]:
    print(df[band].isna().value_counts())

B11
False    12135682
Name: count, dtype: int64
B02
False    12135581
True          101
Name: count, dtype: int64
B12
False    12135682
Name: count, dtype: int64
B08
False    12135682
Name: count, dtype: int64
B04
False    12135653
True           29
Name: count, dtype: int64
B03
False    12135649
True           33
Name: count, dtype: int64


In [57]:
# Extracting month and year
df["year"] = pd.to_datetime(df["date"]).dt.year
df["month"] = pd.to_datetime(df["date"]).dt.month
df.drop('date', axis=1, inplace=True)

# scaling the band values by a factor 0.0001
for band in ["B11", "B02", "B12", "B08", "B04", "B03"]:
    df[band] = df[band] * 0.0001

In [58]:
for band in ["B11", "B02", "B12", "B08", "B04", "B03"]:
    print(df[band].isna().value_counts())

B11
False    12135682
Name: count, dtype: int64
B02
False    12135581
True          101
Name: count, dtype: int64
B12
False    12135682
Name: count, dtype: int64
B08
False    12135682
Name: count, dtype: int64
B04
False    12135653
True           29
Name: count, dtype: int64
B03
False    12135649
True           33
Name: count, dtype: int64


In [59]:
def make_s2_indices(df):

    #### SENTINEL 2

    ## NDVI
    df["NDVI"] = (df["B04"] - df["B03"]) / (df["B04"] + df["B03"])
    temp_df = pd.DataFrame(df["NDVI"])

    ## Atmospherically Resistant Vegetation Index 2
    df["ARVI2"] = -0.18 + 1.17 * df["NDVI"]

    ## Blue-wide dynamic range vegetation index
    df["BWDRVI"] = (0.1 * (df["B08"] - df["B02"])) / (0.1 * (df["B08"] + df["B02"]))

    ## Chlorophyll vegetation index
    df["CVI"] = df["B08"] * (df["B04"] / (df["B03"].pow(2)))

    ## Corrected Transformed Vegetation Index
    temp_df["ndvi+0.5"] = temp_df["NDVI"] + 0.5
    temp_df["ctvi1"] = (temp_df["ndvi+0.5"]) / (temp_df["ndvi+0.5"].abs())
    temp_df["ctvi2"] = np.sqrt(temp_df["ndvi+0.5"].abs())
    df["CTVI"] = temp_df["ctvi1"] * temp_df["ctvi2"]

    ## Enhanced Vegetation Index 2 -2
    df["EVI2"] = 2.5 * ((df["B08"] - df["B04"]) / (df["B08"] + (2.4 * df["B04"]) + 1))

    ## Global Vegetation Moisture Index
    temp_df['gvmi_num'] = (df['B08'] + 0.1) - (df['B12'] + 0.02)
    temp_df['gvmi_den'] = (df['B08'] + 0.1) + (df['B12'] + 0.02)
    df['GVMI'] = temp_df['gvmi_num']/temp_df['gvmi_den']

    ## Modified Soil Adjusted Vegetation Index hyper
    temp_df['hyper_first_term'] = 2 * df['B08'] + 1
    temp_df['hyper_second'] = np.sqrt(temp_df['hyper_first_term'].pow(2) - (8 * (df['B08'] - df['B04'])))
    df['MSVAIhyper'] = 0.5 * (temp_df['hyper_first_term'] - temp_df['hyper_second'])

    ## Modified Triangular Vegetation Index 2
    temp_df['mtvi2_num'] = 1.5 * (1.2 * (df['B08'] - df['B03']) - 2.5 *(df['B04'] - df['B03']))
    temp_df['mtvi2_den'] = np.sqrt((2 * df['B08'] + 1).pow(2) - (6 * df['B08'] - 5*(np.sqrt(df['B04']))) - 0.5)
    df['MTVI2'] = temp_df['mtvi2_num']/temp_df['mtvi2_den']
    # MTVI2 = 1.5*(1.2 * (NIR - Green) - 2.5 * (Red - Green))√((2 * NIR + 1)²-(6 * NIR - 5√(Red)) - 0.5)


    ## Normalized Difference NIR/MIR Modified Normalized Difference Vegetation Index
    df['MNDVI'] = (df['B08'] - df['B12'])/(df['B08'] + df['B12'])

    ## Optimized Soil Adjusted Vegetation Index
    temp_df['Y'] = 0.16
    temp_df['osavi_num'] = df['B08'] - df['B04']
    temp_df['osavi_den'] = df['B08'] + df['B04'] + temp_df['Y']
    df['OSAVI'] = (temp_df['Y'] + 1) * (temp_df['osavi_num']/temp_df['osavi_den'])

    ## Perpendicular Vegetation Index
    pvi_a = 0.149
    pvi_ar = 0.374
    pvi_b = 0.735

    temp_df['pvi_first'] = 1/(np.sqrt(1 + (pvi_a*pvi_a)))
    temp_df['pvi_second'] = df['B08'] - pvi_ar - pvi_b
    df['PVI'] = temp_df['pvi_first'] * temp_df['pvi_second']

    ## Soil and Atmospherically Resistant Vegetation Index
    y = 0.735
    Rr = 0.740
    L = 0.487
    RB = 0.560

    temp_df['sarvi_num'] = df['B08'] - (Rr - y * (RB-Rr))
    temp_df['sarvi_den'] = (df['B08'] + -(Rr - y * (RB-Rr))) + L
    df['SARVI'] = (1+L) * (temp_df['sarvi_num'] / temp_df['sarvi_den'])

    ## Specific Leaf Area Vegetation Index
    df['SLAVI'] = df['B08'] / (df['B04'] + df['B12'])

    ## Transformed Soil Adjusted Vegetation Index 2
    tsavi2_a = 0.419
    tsavi2_b = 0.787
    temp_df['tsavi2_num'] = (tsavi2_a * df['B08']) - (tsavi2_a * df['B04']) - tsavi2_b
    temp_df['tsavi2_den'] = df['B04'] + (tsavi2_a * df['B08']) - (tsavi2_a * tsavi2_b)
    df['TSAVI2'] = temp_df['tsavi2_num'] / temp_df['tsavi2_den']

    ## Weighted Difference Vegetation Index
    wdvi_a = 0.752
    df['WDVI'] = df['B08'] - (wdvi_a * df['B04'])

    ## Wide Dynamic Range Vegetation Index
    df['WDRVI'] = 0.1*(df['B08'] - df['B04'])/(0.1*(df['B08'] + df['B04']))

    return df

df_indices = make_s2_indices(df)
df_indices

,long,lat,species_names,B11,B02,B12,B08,B04,B03,year,month,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MSVAIhyper,MTVI2,MNDVI,OSAVI,PVI,SARVI,SLAVI,TSAVI2,WDVI,WDRVI
0,76.98762370000003,31.571770300000026,chir pine,0.2234,0.0376,0.1084,0.3340,0.0468,0.0739,2019,8,-0.224524,-0.442693,0.797632,2.862223,0.524858,0.496432,0.543385,0.485925,0.488624,0.509946,0.616036,-0.766538,15.603355,2.152062,4.661752,0.298806,0.754202
1,76.98770870000004,31.571733300000062,chir pine,0.2234,0.0240,0.1084,0.3374,0.0248,0.0496,2019,8,-0.333333,-0.570000,0.867183,3.401210,0.408248,0.559445,0.546129,0.561649,0.591280,0.513683,0.694401,-0.763175,16.605351,2.533033,4.010337,0.318750,0.863059
2,76.98786920000003,31.571645600000068,chir pine,0.1826,0.0269,0.0823,0.3458,0.0216,0.0525,2019,8,-0.417004,-0.667895,0.855648,2.709943,0.288090,0.579906,0.626710,0.587000,0.636986,0.615510,0.713068,-0.754867,19.820392,3.328200,3.988417,0.329557,0.882417
3,76.98792120000007,31.572662200000025,chir pine,0.2037,0.0345,0.0971,0.3754,0.0359,0.0699,2019,8,-0.321361,-0.555992,0.831666,2.758255,0.422657,0.580715,0.604726,0.579892,0.603412,0.588995,0.689340,-0.725590,74.635384,2.822556,4.721350,0.348403,0.825432
4,76.98797090000005,31.57163410000004,chir pine,0.1826,0.0247,0.0823,0.3248,0.0245,0.0529,2019,8,-0.366925,-0.609302,0.858655,2.843615,0.364794,0.542606,0.611838,0.542497,0.580178,0.595677,0.683974,-0.775637,13.456736,3.041199,3.908532,0.306376,0.859719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12135677,78.12497830000007,31.25865170000003,rhododendron,0.1781,0.1115,0.1436,0.1827,0.1196,0.1221,2021,11,-0.010343,-0.192102,0.242012,1.465677,0.699755,0.107332,0.266861,0.099708,0.083817,0.119828,0.158330,-0.916186,5.061378,0.694149,5.692750,0.092761,0.208733
12135678,78.12522000000007,31.258880000000033,rhododendron,0.1657,0.1175,0.1431,0.1704,0.1216,0.1230,2021,11,-0.005724,-0.186697,0.183744,1.369597,0.703048,0.083434,0.247520,0.077242,0.063742,0.087081,0.125239,-0.928351,4.856795,0.643748,5.605284,0.078957,0.167123
12135679,78.12614830000007,31.258660000000077,rhododendron,0.1678,0.1113,0.1389,0.1954,0.1222,0.1175,2021,11,0.019608,-0.157059,0.274209,1.729498,0.720838,0.122928,0.300462,0.114727,0.086477,0.169010,0.177789,-0.903624,5.300423,0.748372,6.017877,0.103506,0.230479
12135680,78.12618260000005,31.25882800000005,rhododendron,0.1966,0.1104,0.1481,0.2143,0.1240,0.1219,2021,11,0.008540,-0.170008,0.319988,1.788284,0.713120,0.149315,0.303068,0.140171,0.111598,0.182671,0.210211,-0.884931,5.721906,0.787578,6.460468,0.121052,0.266923


In [60]:
for col in df_indices.columns:
    print(df_indices[col].isna().value_counts())


long
False    12135682
Name: count, dtype: int64
lat
False    12135682
Name: count, dtype: int64
species_names
False    12135682
Name: count, dtype: int64
B11
False    12135682
Name: count, dtype: int64
B02
False    12135581
True          101
Name: count, dtype: int64
B12
False    12135682
Name: count, dtype: int64
B08
False    12135682
Name: count, dtype: int64
B04
False    12135653
True           29
Name: count, dtype: int64
B03
False    12135649
True           33
Name: count, dtype: int64
year
False    12135682
Name: count, dtype: int64
month
False    12135682
Name: count, dtype: int64
NDVI
False    12135620
True           62
Name: count, dtype: int64
ARVI2
False    12135620
True           62
Name: count, dtype: int64
BWDRVI
False    12135581
True          101
Name: count, dtype: int64
CVI
False    12135620
True           62
Name: count, dtype: int64
CTVI
False    12135611
True           71
Name: count, dtype: int64
EVI2
False    12135653
True           29
Name: count, dtype: int64


### Making Monthly Medians

Monthly medians, as expected, have misisng values afte rmedian computations

In [61]:
index_cols = ["long", "lat", "year", "month", "species_names"]
#making monthlu medians
df_month_medians = (
    df_indices.groupby(index_cols)
    .median()
    .reset_index()
)

df_month_medians

,long,lat,year,month,species_names,B11,B02,B12,B08,B04,B03,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MSVAIhyper,MTVI2,MNDVI,OSAVI,PVI,SARVI,SLAVI,TSAVI2,WDVI,WDRVI
0,75.66669580000007,32.25107380000003,2018,2,bambusa bamboo,0.26240,0.14080,0.20730,0.28920,0.16830,0.16120,0.021548,-0.154789,0.345116,1.873063,0.722183,0.178517,0.262612,0.171918,0.134151,0.164955,0.227116,-0.810849,9.022578,0.769968,18.281425,0.162638,0.264262
1,75.66669580000007,32.25107380000003,2018,3,bambusa bamboo,0.28310,0.13900,0.22460,0.31860,0.17640,0.17100,0.015544,-0.161813,0.392483,1.921994,0.718014,0.204080,0.262364,0.197548,0.159463,0.173049,0.251835,-0.781770,12.344106,0.794514,36.628039,0.185947,0.287273
2,75.66669580000007,32.25107380000003,2018,4,bambusa bamboo,0.28970,0.15440,0.23560,0.34180,0.17550,0.17720,-0.004820,-0.185639,0.377670,1.910389,0.703690,0.235820,0.266992,0.228589,0.196250,0.183928,0.284819,-0.758823,18.134563,0.831428,64.981728,0.209824,0.321477
3,75.66669580000007,32.25107380000003,2018,5,bambusa bamboo,0.32590,0.21600,0.26770,0.32420,0.23600,0.23600,0.000000,-0.180000,0.200296,1.373729,0.707107,0.116630,0.191740,0.115042,0.096600,0.095455,0.142061,-0.776231,13.339193,0.643637,-17.821364,0.146728,0.157444
4,75.66669580000007,32.25107380000003,2018,6,bambusa bamboo,0.32320,0.16480,0.26840,0.34100,0.19560,0.19280,0.007209,-0.171565,0.348359,1.794358,0.712186,0.200780,0.209213,0.195647,0.162253,0.119134,0.242125,-0.759614,17.833930,0.734914,-83.208503,0.193909,0.270965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8626389,78.12624330000006,31.258713300000068,2024,8,rhododendron,0.21590,0.11680,0.16170,0.33150,0.11900,0.12700,-0.032520,-0.218049,0.478920,2.445812,0.683725,0.328520,0.407371,0.315368,0.281402,0.344282,0.403767,-0.769010,14.947390,1.180976,9.713553,0.242012,0.471698
8626390,78.12624330000006,31.258713300000068,2024,9,rhododendron,0.19565,0.11470,0.14720,0.32305,0.11240,0.14260,-0.118253,-0.318356,0.475040,1.793129,0.617828,0.330430,0.433184,0.316796,0.313829,0.373414,0.409749,-0.777368,13.982224,1.244150,8.548427,0.238525,0.482899
8626391,78.12624330000006,31.258713300000068,2024,10,rhododendron,0.18175,0.09995,0.12895,0.22385,0.10470,0.10710,-0.011375,-0.193309,0.382070,2.044298,0.699005,0.201748,0.369414,0.189244,0.160179,0.268025,0.282372,-0.875485,6.000114,0.959065,5.628229,0.145116,0.361852
8626392,78.12624330000006,31.258713300000068,2024,11,rhododendron,0.18195,0.12695,0.14900,0.21795,0.13615,0.13635,-0.000657,-0.180768,0.266684,1.596637,0.706638,0.132369,0.305819,0.124763,0.101885,0.187783,0.184541,-0.881321,5.817910,0.764326,7.375042,0.115565,0.230987


### Making Season Medians 

No missing values in seasonal medians

In [62]:
# creating seasons
season_index_cols = ["long", "lat", "season", "species_names"]


conds = [df_month_medians["month"].isin([3, 4, 5]), df_month_medians["month"].isin([10, 11, 12])]
opts = ["summer", "winter"]


df_month_medians["season"] = np.select(conds, opts, default="NA")
df_season_medians = df_month_medians[~(df_month_medians["season"] == "NA")]
df_season_medians = df_season_medians.drop(["year", "month"], axis=1)


df_season_medians = df_season_medians.groupby(season_index_cols).median().reset_index()
df_season_medians

,long,lat,season,species_names,B11,B02,B12,B08,B04,B03,NDVI,ARVI2,BWDRVI,CVI,CTVI,EVI2,GVMI,MSVAIhyper,MTVI2,MNDVI,OSAVI,PVI,SARVI,SLAVI,TSAVI2,WDVI,WDRVI
0,75.66669580000007,32.25107380000003,summer,bambusa bamboo,0.29080,0.144000,0.231400,0.325900,0.17550,0.171200,0.002398,-0.177194,0.384061,1.979580,0.708800,0.225780,0.268881,0.217235,0.179881,0.183928,0.281401,-0.774549,13.678397,0.831428,37.068751,0.190389,0.313389
1,75.66669580000007,32.25107380000003,winter,bambusa bamboo,0.25065,0.138050,0.183250,0.353400,0.15340,0.158100,-0.028147,-0.212932,0.437030,2.175362,0.686915,0.315510,0.393176,0.303823,0.273997,0.334161,0.371065,-0.747350,18.697426,1.102443,26.010812,0.235578,0.416930
2,75.66676390000003,32.25073910000003,summer,bambusa bamboo,0.32060,0.138400,0.252900,0.338200,0.16700,0.165400,0.001119,-0.178691,0.421399,2.062734,0.707897,0.253276,0.237171,0.245149,0.204307,0.145153,0.305154,-0.762384,16.732663,0.799167,28.015505,0.209257,0.345016
3,75.66676390000003,32.25073910000003,winter,bambusa bamboo,0.27175,0.133700,0.195500,0.346350,0.14360,0.153700,-0.037561,-0.223947,0.430745,2.144670,0.680026,0.303925,0.355861,0.290103,0.259030,0.288116,0.371765,-0.754323,14.010840,1.036660,16.317465,0.234902,0.421175
4,75.66686810000004,32.25026770000005,summer,bambusa bamboo,0.34330,0.145500,0.272000,0.338600,0.17840,0.173700,0.009671,-0.168685,0.414938,2.029689,0.713913,0.238532,0.221538,0.232583,0.198892,0.138885,0.280596,-0.761988,16.993831,0.779933,20.908769,0.206843,0.317657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288783,78.12614830000007,31.258660000000077,winter,rhododendron,0.16180,0.128950,0.131850,0.199300,0.11700,0.125200,-0.013914,-0.196279,0.209199,1.515341,0.697091,0.125659,0.314773,0.115946,0.099115,0.196658,0.184029,-0.899767,5.335629,0.744315,5.973100,0.106842,0.238085
288784,78.12618260000005,31.25882800000005,summer,rhododendron,0.23500,0.140125,0.177175,0.293275,0.13340,0.158675,-0.051775,-0.240576,0.311875,1.742588,0.669496,0.205042,0.321752,0.196901,0.169733,0.229229,0.264404,-0.806818,9.690530,0.848251,8.310221,0.176782,0.318488
288785,78.12618260000005,31.25882800000005,winter,rhododendron,0.18360,0.126350,0.143250,0.225050,0.12170,0.131950,-0.019560,-0.202886,0.268772,1.679386,0.693133,0.156620,0.316698,0.145969,0.131382,0.198699,0.226187,-0.874298,5.902077,0.811576,6.544033,0.123481,0.277789
288786,78.12624330000006,31.258713300000068,summer,rhododendron,0.22950,0.142950,0.181575,0.326275,0.14825,0.161575,-0.046930,-0.234908,0.376366,1.896444,0.673092,0.247704,0.352301,0.237653,0.217408,0.274719,0.311434,-0.774178,9.241863,0.952008,14.489785,0.199772,0.364470


### Widening the frame

In [63]:
df_wide = df_season_medians.pivot(index=['long', 'lat', 'species_names'], columns=['season']).reset_index()
new_cols = ["".join(x) for x in df_wide.columns]
df_wide.columns = new_cols
df_wide

,long,lat,species_names,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter
0,75.66669580000007,32.25107380000003,bambusa bamboo,0.290800,0.25065,0.144000,0.13805,0.231400,0.18325,0.325900,0.35340,0.175500,0.15340,0.171200,0.15810,0.002398,-0.028147,-0.177194,-0.212932,0.384061,0.437030,1.979580,2.175362,0.708800,0.686915,0.225780,0.315510,0.268881,0.393176,0.217235,0.303823,0.179881,0.273997,0.183928,0.334161,0.281401,0.371065,-0.774549,-0.747350,13.678397,18.697426,0.831428,1.102443,37.068751,26.010812,0.190389,0.235578,0.313389,0.416930
1,75.66676390000003,32.25073910000003,bambusa bamboo,0.320600,0.27175,0.138400,0.13370,0.252900,0.19550,0.338200,0.34635,0.167000,0.14360,0.165400,0.15370,0.001119,-0.037561,-0.178691,-0.223947,0.421399,0.430745,2.062734,2.144670,0.707897,0.680026,0.253276,0.303925,0.237171,0.355861,0.245149,0.290103,0.204307,0.259030,0.145153,0.288116,0.305154,0.371765,-0.762384,-0.754323,16.732663,14.010840,0.799167,1.036660,28.015505,16.317465,0.209257,0.234902,0.345016,0.421175
2,75.66686810000004,32.25026770000005,bambusa bamboo,0.343300,0.28495,0.145500,0.13645,0.272000,0.20220,0.338600,0.36120,0.178400,0.15125,0.173700,0.16170,0.009671,-0.032607,-0.168685,-0.218151,0.414938,0.448741,2.029689,2.164264,0.713913,0.683661,0.238532,0.304010,0.221538,0.356977,0.232583,0.294028,0.198892,0.267557,0.138885,0.288978,0.280596,0.358286,-0.761988,-0.739635,16.993831,12.035246,0.779933,1.020346,20.908769,21.558192,0.206843,0.251557,0.317657,0.402273
3,75.66719240000003,32.249825100000066,bambusa bamboo,0.327400,0.27330,0.149800,0.13570,0.255900,0.19170,0.343200,0.35615,0.179800,0.14935,0.173000,0.15735,0.001174,-0.037510,-0.178627,-0.223887,0.397944,0.449446,1.984213,2.136440,0.707936,0.680061,0.234842,0.296429,0.231580,0.387440,0.228232,0.285118,0.186260,0.247443,0.140257,0.328659,0.280712,0.380628,-0.757438,-0.744630,18.688164,8.869637,0.783857,1.114056,31.677736,21.970158,0.210621,0.241171,0.314274,0.422492
4,75.66729090000007,32.24962370000003,bambusa bamboo,0.336000,0.27315,0.139100,0.13010,0.263000,0.19265,0.336400,0.35285,0.161500,0.13640,0.164600,0.15030,-0.007733,-0.045869,-0.189047,-0.233667,0.423539,0.449429,2.020029,2.181564,0.701618,0.673890,0.255823,0.324017,0.234204,0.365689,0.246850,0.309672,0.212070,0.283996,0.150380,0.301335,0.308859,0.390062,-0.764164,-0.747894,16.296182,9.351191,0.822092,1.065821,31.849028,13.153691,0.209206,0.249375,0.351275,0.438624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144389,78.12497830000007,31.25865170000003,rhododendron,0.249050,0.17545,0.133900,0.12850,0.186575,0.14170,0.261825,0.20205,0.135225,0.12050,0.151975,0.12890,-0.035770,-0.024648,-0.221851,-0.208838,0.267083,0.223169,1.682089,1.466713,0.681282,0.689428,0.162989,0.110550,0.267316,0.297161,0.155112,0.103468,0.143204,0.093354,0.165023,0.177328,0.217331,0.164141,-0.837925,-0.897047,6.081244,5.403668,0.761374,0.708139,7.888399,6.121914,0.144823,0.099400,0.263308,0.225832
144390,78.12522000000007,31.258880000000033,rhododendron,0.301175,0.16540,0.157125,0.12820,0.219400,0.14240,0.353675,0.19145,0.161150,0.12595,0.181850,0.13715,-0.035894,-0.026513,-0.221996,-0.211020,0.349769,0.164438,1.943044,1.389626,0.681198,0.687968,0.259438,0.086504,0.314578,0.280318,0.251256,0.079495,0.224507,0.077318,0.229324,0.141676,0.309611,0.133086,-0.747078,-0.907531,1.970357,5.222927,0.878303,0.686209,0.052540,6.2907

### Exporting Frames

In [64]:
sent_2_month_medians_path = processed_folder.joinpath("sent_2_month_medians.parquet")
sent_2_season_medians_path = processed_folder.joinpath("sent_2_season_medians.parquet")
sent2_season_medians_widened_path = processed_folder.joinpath("sent2_season_medians_widened.csv")

In [65]:
df_month_medians.to_parquet(sent_2_month_medians_path, index=False)
df_season_medians.to_parquet(sent_2_season_medians_path, index=False)

In [66]:
df_wide.to_csv(sent2_season_medians_widened_path, index=False)